In [49]:
import praw
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import networkx as nx
from tqdm import tqdm
#importing our error
from prawcore.exceptions import Forbidden

In [50]:
client_id = "7jKRRhTGQffOQpxgSfBEpA"
client_secret = "rVl1Tx1gdoQN5uusoeK7arbnR1N8mA"
user_agent = "sicss2022project"
username = "mkalassi"
password = "Kalassina4!"
    
    
reddit = praw.Reddit(client_id=client_id,
                     client_secret=client_secret,
                     user_agent=user_agent,
                     username=username,
                     password=password) 

# data_collector = rdc.DataCollector(
#     client_id=client_id,
#     client_secret=client_secret,
#     user_agent=user_agent,
#     username=username,
#     password=password
# )

## Data Collection from Reddit API

In [114]:
def get_posts(sub_name, n):
    subreddit = reddit.subreddit(sub_name)
    posts_list = [] 
    
    for submission in subreddit.top(limit=n):
        info_list = []
        info_list.append(submission.id)   # Output: the submission's title
        info_list.append(submission.score)
        info_list.append(submission.title)# Output: the submission's score  
        info_list.append(str(submission.author)) 
        info_list.append(submission.num_comments)
        info_list.append(submission.ups)
        info_list.append(submission.downs)
        info_list.append(submission.subreddit)
        posts_list.append(info_list)
    
    a = sorted(posts_list, key=lambda x: x[1], reverse = True)
    posts_df = pd.DataFrame(a, columns = ['id', 'score', 'text','author', 'num_comments','ups','downs','subreddit'])
    return posts_df

def get_comments(subm_id, n): 
    
    com_list = []
    submission = reddit.submission(id = subm_id)
    i = 0
    for comment in submission.comments:
        info_list = []
            # Top 3 comments for each submission
        if i <= n:
            info_list.append(comment.id)
            #info_list.append(comment.score)
            info_list.append(comment.body)
            info_list.append(comment.ups)
            info_list.append(comment.downs)
            info_list.append(comment.author)
            info_list.append(submission.subreddit)
            #info_list.append(submission.comments.list()[i].body)
            i += 1
            com_list.append(info_list)
        else:
            break
    
    a = sorted(com_list, key=lambda x: x[1], reverse = True)
    com_df = pd.DataFrame(a,columns=['id','text','ups','downs','author','subreddit'])
    return com_df  

def get_user_posts(author, n):
    
    redditor = reddit.redditor(author)
    user_posts_list = []
    
    for submission in redditor.submissions.top(limit = n):
        info_list = []
        info_list.append(submission.id)
        info_list.append(submission.score)
        info_list.append(str(submission.author))
        info_list.append(submission.num_comments)
        info_list.append(str(submission.subreddit))
        info_list.append(submission.subreddit_subscribers)
        subspace_reddit = reddit.subreddit(str(submission.subreddit))
        info_list.append(str(subspace_reddit.description))
        user_posts_list.append(info_list)
    
    a = sorted(user_posts_list, key=lambda x: x[1], reverse = True)
    user_posts_df = pd.DataFrame(a,columns=['id','score','author','num_comments','subreddit',"sub_num_subscribers",'subred_desc'])
    user_posts_df['subred_desc']=user_posts_df['subred_desc'].astype('str')
    return user_posts_df

In [45]:
def reddit_scraper1(subname):

    #getting posts under the reddit
    posts = get_posts(subname,None)
    
    #creating empty dataframe for comments under posts
    comments_df=pd.DataFrame(columns=['id','score','text','ups','downs','author','subreddit'])
    for post in tqdm(posts['id']):
        
        try:#comments DF
            x=get_comments(post,100)
            comments_df = pd.concat([comments_df,x],ignore_index=True)
        except Forbidden:
            pass
        
    #creating empty dataframe for users under comments
    users_df=pd.DataFrame(columns=['id','score','author','num_comments','subreddit',"sub_num_subscribers",'subred_desc'])
    for author in tqdm(comments_df['author']):
        try:
            y = get_user_posts(str(author), 50)
            users_df = pd.concat([users_df,y],ignore_index=True)
        except Forbidden:
            pass
    posts.to_csv("posts_final.csv")
    comments_df.to_csv("comments_final.csv")
    users_df.to_csv("users_final.csv")


In [46]:
reddit_scraper('roevwade2022')

100%|██████████| 832/832 [7:09:21<00:00, 30.96s/it]    


## Timetags

In [53]:
get_posts('roevwade2022', 2)
#get_user_posts('QuarterBackground',9)

,id,score,text,author,num_comments,ups,downs,subreddit
0,upq73b,269,BEST of...Signs at Bans Off Our Bodies Marches,QuarterBackground,15,269,0,roevwade2022
1,us1x3f,266,Pro-lifers have spent decades using litigation...,gear-heads,6,266,0,roevwade2022


In [90]:
from datetime import datetime
subreddit = reddit.subreddit('roevwade2022')

l=[]
for submission in subreddit.top(limit=None):
    a=submission.created_utc
    parsed_date = datetime.utcfromtimestamp(a)
    year = parsed_date.year
    month = parsed_date.month
    day = parsed_date.day
    x=[str(submission.id),parsed_date,year,month,day]
    l.append(x)

In [92]:
post_created = pd.DataFrame(l,columns=['post_id','parsed_date','year','month','day'])
post_created.to_csv('post_dates.csv')

In [76]:
x

[datetime.datetime(2022, 5, 14, 4, 6, 52), 2022, 5, 14]

## Other Reddit

In [108]:
def reddit_scraper1(subname):

    #getting posts under the reddit
    posts = get_posts(subname,None)
    posts.to_csv("posts_celebration.csv")
    
    #creating empty dataframe for comments under posts
    comments_df=pd.DataFrame(columns=['id','score','text','ups','downs','author','subreddit'])
    for post in tqdm(posts['id']):
        
        try:#comments DF
            x=get_comments(post,10000)
            comments_df = pd.concat([comments_df,x],ignore_index=True)
        except Forbidden:
            pass
    comments_df.to_csv("comments_celebration.csv")
    #creating empty dataframe for users under comments
#     users_df=pd.DataFrame(columns=['id','score','author','num_comments','subreddit',"sub_num_subscribers",'subred_desc'])
#     for author in tqdm(comments_df['author']):
#         try:
#             y = get_user_posts(str(author), 10)
#             users_df = pd.concat([users_df,y],ignore_index=True)
#         except Forbidden:
#             pass
    
    
#     users_df.to_csv("users_celebration.csv")


In [109]:
reddit_scraper1('RoevWadeCelebration')

100%|██████████| 158/158 [00:36<00:00,  4.34it/s]
